In [0]:
import numpy as np
import gensim
from gensim.models import KeyedVectors
import codecs

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

In [0]:
gensim.models.doc2vec.FAST_VERSION

1

In [0]:
#Generate list of sentences
class MySentences(object):
    def __init__(self, fileName):
        self.fileName = fileName

    def __iter__(self):
        for line in codecs.open(filename=self.fileName, encoding='utf-8'):
            yield line[:-1].split('","')

#Save model after each Epoch
class EpochSaver(CallbackAny2Vec):
    # Callback to save model after each epoch.
    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0

    def on_epoch_end(self, model):
        output_path = '{}_epoch{}.model'.format(self.path_prefix, self.epoch)
        model.save(output_path)
        self.epoch += 1

#Custom Logging
class EpochLogger(CallbackAny2Vec):

    # Callback to log information about training
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [0]:
WORD2VEC_MODEL = 'skip-gram'
WALKS_PATH = '~/Desktop/dataHackSummit/SearchTermRecommendations/termlist.txt'
WORD2VEC_EPOCH_SAVE_PATH = '~/Desktop/dataHackSummit/SearchTermRecommendations/{}/epoch/word2vec'.format(WORD2VEC_MODEL)
FINAL_MODEL_PATH = '~/Desktop/dataHackSummit/SearchTermRecommendations/{}/word2vec{}.model'.format(WORD2VEC_MODEL, WORD2VEC_MODEL)
FINAL_VECTOR_PATH = '~/Desktop/dataHackSummit/SearchTermRecommendations/{}/wordvectors-{}.kv'.format(WORD2VEC_MODEL, WORD2VEC_MODEL)

In [0]:
sentences = MySentences(WALKS_PATH)
epochSaver = EpochSaver(WORD2VEC_EPOCH_SAVE_PATH)
epochLogger = EpochLogger()
callbacks = [epochSaver, epochLogger]

In [ ]:
var_size = 100 #Size of the embeddings. Usually 100-300 does the trick.
var_window = 7 #Max length of our sequences is 15 hence everything will be covered.
var_min_count = 100 #Minimum occurence of search terms
var_sg = 1 # 1 for skip gram 0 for cbow. skip gram gives good vectors for infrequent words.
var_num_iter = 30 # After 25-30 iterations we do not see any major changes in the vectors
# There are other parameters also like hierarchical softmax -
#It is needed when our num of words are more(estimates the costly softmax fn) Not needed in our case

In [0]:
model = Word2Vec(sentences = sentences,
                 size=var_size, window=var_window,
                 min_count=var_min_count, sg=var_sg,
                 workers=8, seed=1234,
                 iter=var_num_iter, callbacks=callbacks)

Epoch #0 start


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end


In [0]:
model.wv.most_similar('beer')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lager', 0.7699299454689026),
 ('vodka', 0.7030935287475586),
 ('gin', 0.6884914040565491),
 ('wine', 0.6875373125076294),
 ('cider', 0.6839544773101807),
 ('corona', 0.6719756126403809),
 ('budweiser', 0.6610127091407776),
 ('white wine', 0.6550659537315369),
 ('prosecco', 0.6474155783653259),
 ('rose wine', 0.6439282298088074)]

In [0]:
model.predict_output_word(['chicken','spice','soup', 'vegetable'], topn = 20)

[('noodles', 0.0029194446),
 ('fish', 0.002858851),
 ('curry sauce', 0.002659693),
 ('soup', 0.0024856278),
 ('pasta sauce', 0.0024547083),
 ('salmon', 0.0024233577),
 ('meatball', 0.002419044),
 ('peas', 0.0023551204),
 ('curry', 0.002312012),
 ('fajita', 0.0022828763),
 ('rice', 0.0021993315),
 ('beef', 0.0021332463),
 ('steak', 0.0021258784),
 ('stir fry', 0.0021248306),
 ('veg', 0.0021211605),
 ('vegetables', 0.0021107865),
 ('yorkshire puddings', 0.0020970942),
 ('pork', 0.0020772126),
 ('beans', 0.002056384),
 ('mash', 0.002048504)]

In [0]:
model.wv.save(FINAL_VECTOR_PATH)
model.save(FINAL_MODEL_PATH)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Trying Harder

In [1]:
from keras.layers import Dense, Activation, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, CSVLogger
from keras.metrics import top_k_categorical_accuracy, sparse_top_k_categorical_accuracy
from keras.utils.np_utils import to_categorical
import functools

Using TensorFlow backend.


In [2]:
WORD2VEC_MODEL = 'skip-gram'
DROPOUT = 0.5 #Dropout Percentage
NEPOCHS = 50 # No of epochs to train RNN
LAYERS = 2 # No of layers in RNN
TRAD_DROPOUT = 0.1 # Traditional Dropout
RECUR_DROPOUT = 0.5 # Reccurent Dropout
NVOCAB  = 10000 # No of words in the Vocabulary 
EMBEDDING_DIM = 400 #Length of word2vec dimention
NHIDDEN = 300   # size of hidden layer(s) 
N = 5        # Length of Sequence to RNN [Input + Output]
RNN_CLASS = LSTM # type of RNN to use - SimpleRNN, LSTM, or GRU
BATCH_SIZE = 256 #size of batch to use for training
PATIENCE = 10        # stop after this many epochs of no improvement                    
VALIDATION_SPLIT = 0.01 # percent of training data to use for validation (0.01 ~10k tokens)
LOSS_FN    = 'sparse_categorical_crossentropy' # Loss Function used for trainning
OPTIMIZER        = 'adam'    # optimizing algorithm to use (sgd, rmsprop, adam, adagrad, adadelta, adamax, nadam)  
INITIALIZER      = 'uniform' # random weight initializer (uniform, normal, lecun_uniform, glorot_uniform [default])
SEED             = 0         # random number seed                                                                  

In [ ]:
EMBEDDING_MATRIX_DIR = '/content/drive/My Drive/SearchTermRecommendations/Sequence1/embeddingMatrix.npy'
X_TRAIN_DIR = '/content/drive/My Drive/SearchTermRecommendations/Sequence/inputSequence_Train.npy'
Y_TRAIN_DIR = '/content/drive/My Drive/SearchTermRecommendations/Sequence/outputSequence_Train.npy'
X_TEST_DIR = '/content/drive/My Drive/SearchTermRecommendations/Sequence/inputSequence_Test.npy'
Y_TEST_DIR = '/content/drive/My Drive/SearchTermRecommendations/Sequence/outputSequence_Test.npy'
MODEL_SAVE_FILE_PATH = '/content/drive/My Drive/SearchTermRecommendations/RNNModel_2_SKIPGRAM_1024/model-{epoch:02d}-{val_top20_acc:.2f}.hdf5'
CSV_LOGGER_SAVE_FILE_PATH = '/content/drive/My Drive/SearchTermRecommendations/ModelTrainningLogs/rnn.csv'

In [ ]:
embeddingMatrix = np.load(EMBEDDING_MATRIX_DIR)
X_TRAIN = np.load(X_TRAIN_DIR)
Y_TRAIN = np.load(Y_TRAIN_DIR)
X_TEST = np.load(X_TEST_DIR)
Y_TEST = np.load(Y_TEST_DIR)
NVOCAB,EMBEDDING_DIM = embeddingMatrix.shape

In [ ]:
# --------------------------------------------------------------------------------
# Build Model                                                                     
# --------------------------------------------------------------------------------
                                                                                  
model = Sequential()                                                              
                                                                                  
# embedding layer                                                                 
embedding_layer = Embedding(input_dim=NVOCAB, output_dim=EMBEDDING_DIM, input_length=N-1, weights=[embeddingMatrix])                        
model.add(embedding_layer)

if LAYERS==1:                                                             
    model.add(RNN_CLASS(NHIDDEN, kernel_initializer=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT))
    #model.add(Dropout(DROPOUT))
                                           
elif LAYERS==2:                                                           
    model.add(RNN_CLASS(NHIDDEN, kernel_initializer=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT, return_sequences=True))
    #model.add(Dropout(DROPOUT))
    model.add(RNN_CLASS(NHIDDEN, kernel_initializer=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT))
    #model.add(Dropout(DROPOUT))
    
elif LAYERS==3:
    model.add(RNN_CLASS(NHIDDEN, init=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT, return_sequences=True))
    #model.add(Dropout(DROPOUT))
    model.add(RNN_CLASS(NHIDDEN, init=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT, return_sequences=True))
    #model.add(Dropout(DROPOUT))
    model.add(RNN_CLASS(NHIDDEN, init=INITIALIZER, dropout = TRAD_DROPOUT, recurrent_dropout = RECUR_DROPOUT))
    #model.add(Dropout(DROPOUT))
    

# output layer - convert nhidden to nvocab
model.add(Dense(NVOCAB-1)) # The padded vocab not included in output layer

# convert nvocab to probabilities - expensive 
model.add(Activation('softmax'))

# compile the model ~ 1 sec
top10_acc = functools.partial(sparse_top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'
top20_acc = functools.partial(sparse_top_k_categorical_accuracy, k=20)
top20_acc.__name__ = 'top20_acc'
metrics = [top10_acc, top20_acc] # loss is always the first metric returned from the fit method
model.compile(loss=LOSS_FN, optimizer=OPTIMIZER, metrics=metrics)

# Define the callbacks that will be required for fitting the model
early_stopping = EarlyStopping(monitor='top20_acc', patience=PATIENCE)
checkpoint = ModelCheckpoint(MODEL_SAVE_FILE_PATH, monitor='top20_acc', save_best_only=True, mode='max')
csvlogger = CSVLogger(CSV_LOGGER_SAVE_FILE_PATH)
callbacks = [checkpoint, early_stopping, csvlogger]

In [0]:
# Define the callbacks that will be required for fitting the model
early_stopping = EarlyStopping(monitor='top20_acc', patience=PATIENCE)
checkpoint = ModelCheckpoint(MODEL_SAVE_FILE_PATH, monitor='top20_acc', save_best_only=True, mode='max')
csvlogger = CSVLogger(CSV_LOGGER_SAVE_FILE_PATH)
callbacks = [checkpoint, early_stopping, csvlogger]


In [0]:
#train the model
history = model.fit(X_TRAIN, Y_TRAIN, batch_size=BATCH_SIZE, 
                    epochs=NEPOCHS,validation_split=VALIDATION_SPLIT, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 6732799 samples, validate on 68009 samples
Epoch 1/50
6732799/6732799 [==============================] - 662s 98us/step - loss: 4.6838 - top10_acc: 0.3846 - top20_acc: 0.4810 - val_loss: 4.5808 - val_top10_acc: 0.3993 - val_top20_acc: 0.4972
Epoch 2/50
6732799/6732799 [==============================] - 660s 98us/step - loss: 4.5659 - top10_acc: 0.4025 - top20_acc: 0.5018 - val_loss: 4.5590 - val_top10_acc: 0.4038 - val_top20_acc: 0.5025
Epoch 3/50
6732799/6732799 [==============================] - 659s 98us/step - loss: 4.5403 - top10_acc: 0.4065 - top20_acc: 0.5066 - val_loss: 4.5500 - val_top10_acc: 0.4045 - val_top20_acc: 0.5044
Epoch 4/50
6732799/6732799 [==============================] - 761s 113us/step - loss: 4.5242 - top10_acc: 0.4087 - top20_acc: 0.5095 - val_loss: 4.5461 - val_top10_acc: 0.4065 - val_top20_acc: 0.5051
Epoch 5/50
6732799/6732799 [==============================] - 859s 128us/step - loss: 4.5117 - top10_ac